# Using itertools

http://www.blog.pythonlibrary.org/2016/04/20/python-201-an-intro-to-itertools/

In [ ]:
"""Index positions and amounts of all combinations adding up to the target amount"""

from itertools import combinations

dif = 17000
amounts = [100000 , 65000 , 7000 , 3000 , 17000 , 10000 , 3000 , 4000]
amounts = [(idx, amount) for idx, amount in enumerate(amounts, start=1)]
totals = (comb for qty in range(2, len(amounts) + 1)
          for comb in combinations(amounts, qty)
          if sum(n[1] for n in comb) == dif
              )
for total in totals:
    print(*total, sep=", ")

In [ ]:
#itertools

from itertools import cycle,islice, count, repeat, accumulate, compress, chain

for num in count(5):  # infinite counter
    if num == 10:  # need an exit plan
        break
    print(num, end="; ")
print()
for num in islice(count(5), 5):  # limits iterations
    print(num, end="; ")
print()
for letter in islice(cycle('xyz'), 5):
    print(f'{letter}', end="; ")
print()   
for string in repeat('Stuart', 5):  # boring repititions
    print(f'Hi {string}', end="; ")
print()
for num in accumulate(islice(count(0), 10)):
    print(num, end="; ")
print()

nums = [1,2,3]
names = 'fred', 'bob'
for thing in chain(nums, names):
    print(thing, end="; ")
print()

bools = [True, False, True]
for num in compress(nums, bools):
    print(num, end='; ')
print()

In [ ]:
from itertools import combinations, filterfalse

def off_target(nums):  # combinations that don't equal target
    return not sum(n for _, n in nums) == dif

def output_matches(totals, amounts):
    print(f"\n\nAmounts that sum to {dif}:\n"
          f"  (from: {', '.join(str(n) for n in amounts)})\n")
    for total in totals:
        for idx, num in total:
            print(f'{idx:4}: {num:5}', end='  ')
        print()


difs = ((17000, [100000 , 65000 , 7000 , 3000 , 17000 , 10000 , 3000 , 4000]), # original
        (10000, [100000 , 65000 , 7000 , 3000 , 17000 , 10000 , 3000 , 4000]), # test
       )

for dif, amounts in difs:
    amounts_idx = [(idx, amount) for idx, amount in enumerate(amounts, start=1)]
    totals = (comb for qty in range(2, len(amounts) + 1)
                      for comb in filterfalse(off_target,
                            (combinations(amounts_idx, qty))))
    output_matches(totals, amounts)

# RPN Calculator using Class

https://www.reddit.com/r/learnpython/comments/fhk6nx/any_way_to_define_a_dictionary_of_methods_within/

In [ ]:
class Calculator:

    GUIDE = ''  # placeholder for init

    def __init__(self):
        self.clear()
        self.degrees = True
        self.scimode = False
        self.rpn = True
        self.msg = "h(elp) q(uit)"
        self.err = ""
        self.prompt = "> "
        print(Calculator.GUIDE)

    def __str__(self):
        elements = [self.err, self.msg, f'{self.stack}', self.prompt]
        self.display = ' | '.join(elements)
        return self.display

    @staticmethod
    def is_quit(text):
        return text.lower() in ('q', 'quit', 'exit', 'x')

    def stack_op(self, text):
        if text.lower() in Calculator.OPS:
            self.stack.append(Calculator.OPS[text.lower()][-1])
            return True
        return False

    def stack_num(self, text):
        num = None
        try:
            num = float(text)
            num = int(text)
        except ValueError:
            pass
        if num is None:
            return False
        self.stack.append(num)
        return True                

    def clear(self):
        self.display = ""
        self.memory = 0
        self.stack = []

    def clear_error(self):
        self.err = ""

    def set_msg(self, msg=""):
        self.msg = msg

    def enough_nums(self, required=2):
        """confirm if stack holds sufficient numbers for an operation"""
        if len(self.stack) >= required:
            for n in range(required):
                if not isinstance(self.stack[-1-n], (int, float)):
                    return False
            return True
        return False    

    def entry(self):
        while True:
            response = input(f'{self}')
            self.clear_error()
            if self.is_quit(response):
                return None
            elif self.stack_op(response):
                return True
            elif self.stack_num(response):
                return False
            self.err = 'ERR (unknown)'

    def op_add(self):
        if self.enough_nums():
            print(self.stack)
            self.stack.append(self.stack.pop() + self.stack.pop())
        else:
            self.err = f'ERR (not enough nums)'

    def op_sub(self):
        if self.enough_nums():
            self.stack.append(- self.stack.pop() + self.stack.pop())
        else:
            self.err = f'ERR (not enough nums)'

    def op_mul(self):
        if self.enough_nums():
            self.stack.append(self.stack.pop() * self.stack.pop())
        else:
            self.err = f'ERR (not enough nums)'

    def op_div(self):
        if self.enough_nums():            
            if not self.stack[-1] == 0:
                self.stack.append(self.stack.pop(-2) / self.stack.pop())
            else:
                self.err = f'ERR (div by 0 not allowed) {self.stack}'
        else:
            self.err = f'ERR (not enough nums)'

    def op_del1(self):
        if self.stack:
            self.stack.pop()

    def op_whole(self):
        if self.enough_nums(1):
            self.stack.append(int(self.stack.pop()))
        else:
            self.err = f'ERR (not enough nums)'

    def op_round(self):
        if self.enough_nums():
            self.stack.append(round(self.stack.pop(-2), self.stack.pop()))
        else:
            self.err = f'ERR (not enough nums)'

    def op_swap(self):
        if self.enough_nums(2):
            self.stack.extend((self.stack.pop(), self.stack.pop()))
        else:
            self.err = f'ERR (not enough nums)'

    def op_help(self):
        print(Calculator.GUIDE)

    def work(self):
        while True:
            status = self.entry()
            if status is None:
                return
            elif status:
                op = self.stack.pop()
                op(self)

    OPS = {'+': ('add two numbers', op_add),
        '-': ('subtract top from next down', op_sub),
        '*': ('multipple two numbers', op_mul),
        '/': ('divide 2 down by top', op_div),
        'int': ('truncate top to integer', op_whole),
        'round': ('round 2 down to top decimal places', op_round), 
        'h': ('display hel', op_help),
        'del': ('remove top number', op_del1),
        'swap': ('swap top two numbers', op_swap),
        'q': ('quit Calculator', None),
        }

    DETAILS = ''.join(f'\t{code:5} {desc}\n' for code, (desc, _) in OPS.items()) 

    GUIDE = f'Welcome to our simple RPN Calculator\n\n' \
            f'Operators/Commands:\n{DETAILS}\n'


calc1 = Calculator()
calc1.work()

I realise you've been given a few options already, but a slightly different approach is to put the method names in the dict:

In [ ]:
class Calculator:
    method_aliases = {'+': 'add', 'del': 'del1'} # and so on

    def add(self):
        ...

    def del1(self):
        ...

    def get_method(self, string):
        return getattr(self, self.method_aliases[string])

calculator = Calculator()
calculator.get_method('+')() # this calls calculator.add()

This way you can populate the dict anywhere, regardless of whether the methods have been defined yet or not. This also plays nicely with subclasses. For example if you have a subclass that defines its own add method, that will be called instead of the parent class's method, without having to give the subclass a different method_aliases dict.

Though if you're going to do this kind of thing often, it may be nicer to do some magic with decorators and/or metaclasses to get a nice API like the following:

In [ ]:
@alias_class
class Calculator:
    @alias('+')
    def add(self):
        ...

calculator = Calculator()
calculator['+']()

There are a number of ways of coding this up, depending on the exact semantics you want.

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

p = np.linspace(0, 20, 100)
plt.plot(p, np.sin(15*p))
plt.show()



In [ ]:
for i in range(10, 100, 2):
    print(i, end=', ')
print()

In [ ]:
    def inner_sort(record):
        key, value = record
        return value['Name']  + value['middlename']

    example = {"one": {"Name": "bbb", "middlename": "bbb"},
            "two":{"Name": "aaa", "middlename": "aaa"},
            "three":{"Name": "ddd", "middlename": "fff"},
            "four":{"Name": "ccc", "middlename": "ggg"},
            "five":{"Name": "ddd", "middlename": "ccc"},
                }

    contacts = sorted(example.items(), key = inner_sort)
    print(contacts)

In [ ]:
def fac_list(n, lst=None):
    if lst is None:
        lst = []
        print('defined list')
    if n == 1:
        lst.append(n)
        i = 1
        print('oner')
    else:
        print('calling with one fewer', n-1)
        i = n * fac_list(n-1, lst)
        lst.append(i)
    return i

fac_list(5)
lst

In [ ]:
    def get_num(prompt, lower=None, higher=None):
        """return validated integer/float input between lower and higher values inclusive"""
        while True:
            response = input(prompt)
            try:
                num = int(response)
            except ValueError:
                print('Expected a whole number')
            else:
                if lower and num < lower:
                    print('Number is too low')
                elif higher and num > higher:
                    print('Number is too high')
                else:
                    return num
            finally:
                print('Thank you for your input.\n')

            print('Please try again.')


    n1, n2 = get_num('Any int: '), get_num('Int between 10 and 20 inc: ', 10, 20)
    print(n1, n2, n1*n2)

In [ ]:
    from random import randint
    
    class Character:

        def __init__(self, name, kind):
            self.name = name
            self.kind = kind
            self.health = randint(1, 50)
            self.strength = randint(1, 50 - self.health)
            self.dexterity = randint(1, 50 - self.health - self.strength)
            self.wisdom = randint(1, 50 - self.health - self.strength - self.dexterity)

    player1 = Character('Barry', 'wizard')
    print(vars(player1))   # define __str__ and __repr__ methods for decent print

In [ ]:
res = True
row = 'abcdef'
string = 'ktbjqcwxyzaf'
list2 = ['x','y','z']
list3 = []

if res:
    list2.extend(row)
    for j in (letter for letter in string:
        x=list2.index(j)
        list3.append(f'>{x+1}< ')

In [ ]:
    from random import randint

    def rolldice():
        return randint(1,6), randint(1,6)

    def game():
        rounds = 1
        while True:
            print(f'Round number {rounds}')
            user, comp = rolldice(), rolldice()
            print(f'You rolled: {user[0]} and {user[1]}')
            print(f'I rolled:  {comp[0]} and {comp[1]}')
            rounds += 1
            user_total, comp_total = sum(user), sum(comp)
            if user_total == comp_total:
                print('Draw')
            elif user_total > comp_total:
                print('You win')
            else:
                print('I win')

            if input('Press enter to roll again, or q to quit'):
                break
            
    print('This program rolls dice!')
    game()

In [ ]:
def f(n):
    return abs(100-n) <= 10 or abs(200-n) <=10

for test in 89, 91, 105, 111, 189, 192, 200, 208, 211:
    print(f'{test}: {f(test)}')

In [ ]:
test = list("xXfuUuuF")
sorted(test, key=lambda letter: (letter.upper(), letter.islower()))

In [ ]:
import Test

# sorting strings with mixed case

In [ ]:
def find_children(pattern):
    return ''.join(sorted(pattern, key=lambda letter: (letter.upper(), letter.islower())))


tests = [("abBA", "AaBb"), 
        ("AaaaaZazzz", "AaaaaaZzzz"),
        ("CbcBcbaA", "AaBbbCcc"),
        ("xXfuUuuF", "FfUuuuXx"),
        ("", ""),
        ]

for test, answer in tests:
    result = find_children(test)
    print(f'{test} -> {result}   {"pass" if result == answer else "fail"}')